In [1]:
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
import pandas as pd
import pickle

In [2]:
data=pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)

In [4]:
label_encoder_gender=LabelEncoder()
data['Gender']=label_encoder_gender.fit_transform(data['Gender'])

In [5]:
label_encoder_geo=OneHotEncoder()
geo_encoder=label_encoder_geo.fit_transform(data[['Geography']]).toarray()

In [6]:
geo_encoder_df=pd.DataFrame(geo_encoder,columns=label_encoder_geo.get_feature_names_out(['Geography']))
data=pd.concat([data.drop('Geography',axis=1),geo_encoder_df],axis=1)

In [7]:
geo_encoder_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [8]:
X=data.drop('EstimatedSalary',axis=1)
y=data['EstimatedSalary']
X

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,1,0.0,1.0,0.0


In [9]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [10]:
scaler=StandardScaler()

In [11]:
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [12]:
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)

with open('label_encoder_geo.pkl','wb') as file:
    pickle.dump(label_encoder_geo,file)
    
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)      

In [13]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [15]:
model = Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)),
    Dense(32,activation='relu'),
    Dense(1)#by default linear activation function
])

model.compile(optimizer='adam',loss='mean_absolute_error',metrics=['mae'])

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, 64)                832       
                                                                 
 dense_2 (Dense)             (None, 32)                2080      
                                                                 
 dense_3 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [18]:
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

log_dir="regressionlogs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback=TensorBoard(log_dir=log_dir, histogram_freq=1)

In [20]:
early_stopping_callback=EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [21]:
history=model.fit(
    X_train,y_train,
    validation_data=(X_test, y_test),
    epochs=100,
    callbacks=[early_stopping_callback,tensorboard_callback]
)

Epoch 1/100


250/250 [==============================] - 1s 3ms/step - loss: 100370.0391 - mae: 100370.0391 - val_loss: 98490.0156 - val_mae: 98490.0156
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 99522.2266 - mae: 99522.2266 - val_loss: 96768.4531 - val_mae: 96768.4531
Epoch 3/100
250/250 [==============================] - 0s 2ms/step - loss: 96519.7266 - mae: 96519.7266 - val_loss: 92370.3828 - val_mae: 92370.3828
Epoch 4/100
250/250 [==============================] - 0s 2ms/step - loss: 90677.6250 - mae: 90677.6250 - val_loss: 85158.7656 - val_mae: 85158.7656
Epoch 5/100
250/250 [==============================] - 0s 2ms/step - loss: 82302.2422 - mae: 82302.2422 - val_loss: 76068.4375 - val_mae: 76068.4375
Epoch 6/100
250/250 [==============================] - 0s 2ms/step - loss: 72718.7422 - mae: 72718.7422 - val_loss: 67038.9922 - val_mae: 67038.9922
Epoch 7/100
250/250 [==============================] - 0s 2ms/step - loss: 63828.8945 - mae: 63828.894

In [22]:
%load_ext tensorboard

In [24]:
%tensorboard --logdir regressionlogs/fit

Reusing TensorBoard on port 6006 (pid 16132), started 0:03:50 ago. (Use '!kill 16132' to kill it.)

In [25]:
## Evaluating the model 
test_loss,test_mae=model.evaluate(X_test,y_test)
print(f'test MAE :{test_mae}')

63/63 [==============================] - 0s 905us/step - loss: 50359.1133 - mae: 50359.1133
test MAE :50359.11328125


In [26]:
model.save('regression_model.h5')

c:\Development\Project3\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
